In [1]:
import numpy as np
import pandas as pd

## Constants

In [59]:
VOCABULARY_SIZE = 2500

TRAINING_DATA = 'SpamData/02_Training/train-data.txt'
TESTING_DATA = 'SpamData/02_Training/test-data.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-ham.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'


In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA, delimiter=' ',dtype='int')
sparse_test_data = np.loadtxt(TESTING_DATA, delimiter=' ',dtype='int')

In [4]:
sparse_train_data.shape

(257864, 4)

In [5]:
sparse_test_data.shape

(117573, 4)

In [6]:
sparse_train_data[:5] # columns =  doc_id, word_id, category, occurence

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [7]:
sparse_test_data[:5]

array([[8, 2, 1, 1],
       [8, 3, 1, 4],
       [8, 4, 1, 2],
       [8, 5, 1, 1],
       [8, 6, 1, 2]])

In [8]:
print(f'Number of rows in training file : {sparse_train_data.shape[0]}')
print(f'Number of rows in testing file : {sparse_test_data.shape[0]}')

Number of rows in training file : 257864
Number of rows in testing file : 117573


In [9]:
print(f' Number of emails in training file : {np.unique(sparse_train_data[: , 0]).size}')
print(f' Number of emails in testing file : {np.unique(sparse_test_data[: , 0]).size}')

 Number of emails in training file : 4013
 Number of emails in testing file : 1724


## Creating empty dataframe

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0 , VOCABULARY_SIZE))

In [11]:
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [12]:
len(column_names)

2502

In [13]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [14]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)

In [15]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
full_train_data.fillna(value=0,inplace=True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Creating Full Matrix from sparse matrix

In [17]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    '''
    returns full matrix which contains occurences of each word in vocab in single row 
    sparse_matrix -- input matrix (np array)
    nr_words -- size of vocabulary(2500)
    doc_idx -- DOC_ID index in sparse matrix(0)
    word_idx -- 'WORD_ID' index in sparse_matrix(1)
    cat_idx -- 'CATEGORY' index (2)
    freq_idx -- 'OCCURENCES' index (3)
    '''
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0 , VOCABULARY_SIZE))
    doc_id_names = np.unique(sparse_train_data[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_train_data.shape[0]):  # iterating through all rows in sparse matrix and extracting word occurences
        doc_nr = sparse_train_data[i][doc_idx] # doc_idx = 0 first column
        word_id = sparse_train_data[i][word_idx]
        label = sparse_train_data[i][cat_idx]
        occurence = sparse_train_data[i][freq_idx]
        
        full_matrix.at[doc_nr,'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr,'CATEGORY'] = label
        full_matrix.at[doc_nr,word_id] = occurence
        
    full_matrix.set_index('DOC_ID',inplace=True)  
    return full_matrix

In [18]:
%%time
full_train_data = make_full_matrix(sparse_train_data,VOCABULARY_SIZE)

Wall time: 17.4 s


In [19]:
full_train_data.head()  # most frequent words are at begining

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training Naive Bayes Classifier

## calculating probability of spam 

In [21]:
full_train_data['CATEGORY'].size

4013

In [22]:
full_train_data['CATEGORY'].sum()

1248

In [23]:
spam_prob = full_train_data['CATEGORY'].size / full_train_data['CATEGORY'].sum()
print(' Probability os email being spam ' ,round(spam_prob,3))

 Probability os email being spam  3.216


## Total number of words / tokens

In [24]:
full_train_features = full_train_data.loc[:,full_train_data.columns != 'CATEGORY']
full_train_features.head()  #excluding category

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [26]:
email_lengths[:4]

DOC_ID
0     87
1     53
2     40
3    182
dtype: int64

In [27]:
total_wordcounts = email_lengths.sum()
total_wordcounts

428341

## Number of tokens/words in spam or ham emails

In [28]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [29]:
spam_wordcount = spam_lengths.sum()
spam_wordcount

175878

In [30]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [31]:
ham_wordcount = ham_lengths.sum()
ham_wordcount

252463

In [32]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]  #toal emails = spam + ham emails

0

In [33]:
total_wordcounts - spam_wordcount - ham_wordcount  

0

In [34]:
print(f'Avg number of words in spam emails : {spam_wordcount / spam_lengths.shape[0]}')

Avg number of words in spam emails : 140.9278846153846


In [35]:
print(f'Avg number of words in ham emails : {ham_wordcount / ham_lengths.shape[0]}')

Avg number of words in ham emails : 91.30669077757685


## Summing the Tokens Occuring in spam

In [36]:
full_train_features.shape

(4013, 2500)

In [38]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
train_spam_tokens.shape

(1248, 2500)

In [41]:
# Calculating number of time each word appeared in all spam emails

summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 # +1 to avoid zero by error while calculating probability
summed_spam_tokens.shape    # axis = 0 to sum columns

(2500,)

In [42]:
summed_spam_tokens.tail()

2495    22
2496     3
2497     1
2498    20
2499     0
dtype: int64

In [43]:
summed_spam_tokens.head()

0    2168
1     933
2    1216
3    2021
4    1210
dtype: int64

## Summing tokens occuring in ham emails

In [44]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
# Calculating number of time each word appeared in all ham emails

summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 # +1 to avoid zero by error while calculating probability
summed_ham_tokens.shape    # axis = 0 to sum columns

(2500,)

In [46]:
summed_ham_tokens.tail()

2495     9
2496    36
2497     1
2498    10
2499    25
dtype: int64

In [47]:
summed_ham_tokens.head()

0    5483
1    2588
2    2044
3     938
4    1611
dtype: int64

## p( token | spam ) : Probabilty that token ocuurs given email is spam

In [48]:
prob_tokens_spam = summed_spam_tokens / (spam_wordcount + VOCABULARY_SIZE)  #devide by VOCABULARY_SIZE coz we added 1 to 
                                                                                    #summed_spam tokens

In [50]:
prob_tokens_spam[:5]

0    0.012154
1    0.005230
2    0.006817
3    0.011330
4    0.006783
dtype: float64

In [51]:
prob_tokens_spam.sum()

0.9859848187556761

## p( token | nonspam ) : Probabilty that token ocuurs given email is non-spam

In [52]:
prob_tokens_ham = summed_ham_tokens / (ham_wordcount + VOCABULARY_SIZE)

In [53]:
prob_tokens_ham.sum()

1.0000000000000002

## p(token) : probability that token occurs

In [54]:
prob_token_all = full_train_features.sum(axis=0) / total_wordcounts

In [55]:
prob_token_all

0       0.017860
1       0.008218
2       0.007608
3       0.006906
4       0.006584
          ...   
2495    0.000070
2496    0.000089
2497    0.000002
2498    0.000068
2499    0.000056
Length: 2500, dtype: float64

In [56]:
prob_token_all.sum()

1.0

## Saving trained model

In [58]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_token_all)

## Prepare test data

In [60]:
sparse_test_data.shape

(117573, 4)

In [61]:
%%time
full_test_data = make_full_matrix(sparse_test_data,nr_words=VOCABULARY_SIZE)

Wall time: 17.2 s


In [63]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [64]:
np.savetxt(TEST_TARGET_FILE,y_test)
np.savetxt(TEST_FEATURE_MATRIX,X_test)